In [1]:
import sagemaker
from sagemaker.pytorch import PyTorch

In [2]:
sagemaker_session = sagemaker.Session()
bucket = sagemaker_session.default_bucket()
role = sagemaker.get_execution_role()

In [ ]:
!rm -rf sourcedir.tar.gz
!tar czvf sourcedir.tar.gz *

In [ ]:
source_dir = 's3://{0}/{1}/source/'.format(bucket, 'detr')
!aws s3 cp sourcedir.tar.gz $source_dir
source_dir = '{0}sourcedir.tar.gz'.format(source_dir)
!rm -f sourcedir.tar.gz

In [ ]:
!touch dummy
!tar czvf model.tar.gz dummy
assets_dir = 's3://{0}/{1}/assets/'.format(bucket, 'detr')
model_data = 's3://{0}/{1}/assets/model.tar.gz'.format(bucket, 'detr')
!aws s3 cp model.tar.gz $assets_dir
!rm -f dummy model.tar.gz

In [6]:
model_name = None
entry_point = 'inference.py'
framework_version = '1.9.0'
py_version = 'py38'
model_environment = {}

In [7]:
from sagemaker.pytorch.model import PyTorchModel

model = PyTorchModel(
    name = model_name,
    model_data = model_data,
    entry_point = entry_point,
    source_dir = source_dir,
    role = role,
    framework_version = framework_version, 
    py_version = py_version,
    env = model_environment
)

In [8]:
endpoint_name = None
instance_type = 'ml.g4dn.xlarge'
instance_count = 1

In [ ]:
predictor = model.deploy(
    endpoint_name = endpoint_name,
    instance_type = instance_type, 
    initial_instance_count = instance_count
)

In [ ]:
!wget http://images.cocodataset.org/val2017/000000039769.jpg -O test.jpg

In [ ]:
bytes  = open('test.jpg', 'rb').read()

In [ ]:
from sagemaker.serializers import DataSerializer 
from sagemaker.deserializers import JSONDeserializer

predictor.serializer = DataSerializer()
predictor.deserializer = JSONDeserializer()

predictor.predict(bytes, initial_args={"ContentType":"image/png"})

In [ ]:
predictor.delete_endpoint()